# Modelo de motor DC

El par $\tau_r$ en el eje de un motor DC con constante de par $K_{tr}$, momento de inercia del rotor $I_r$, y coeficiente de rozamiento viscoso $b_r$, cuando circula una corriente $i$ por sus bobinados y gira a una velocidad $\omega_r$, será:

$$
\begin{equation}
    \tau_r = K_{tr}i - I_r\dot\omega_r - b_r\omega_r
\end{equation}
$$

Al introducir una reductora con relación de transmisión $n$ y rendimiento $\eta$, la velocidad de giro a la salida $\omega_g$ se reduce en un factor $n$. El par, por el contrario, es multiplicado por el mismo factor, además de verse afectado por las pérdidas mecánicas en los engranajes, representadas a través del rendimiento $\eta$:

$$
\begin{equation}
    \omega_g = \frac{\omega_r}{n} \qquad
    \tau_g = \eta n\tau_r
\end{equation}
$$

Sustituyendo ambas expresiones en la primera ecuación, se obtiene la relación entre par y velocidad a la salida de la reductora:

$$
\begin{equation}
    \tau_g = \eta nK_{tr}i - \eta n^2I_r\dot\omega_g - \eta n^2b_r\omega_g
\end{equation}
$$

En esta expresión, es fácil identificar los parámetros equivalentes del conjunto motor-reductora, visto desde el lado de la carga:

$$
\begin{equation}
    K_{tg} = \eta nK_{tr} \qquad
    b_g = \eta n^2b_r     \qquad
    I_g = \eta n^2I_r
\end{equation}
$$

Del mismo modo, se puede proceder con la ecuación del circuito eléctrico. El voltaje $V$ en los bornes del motor es:

$$
\begin{equation}
    V = L\frac{\mathrm{d}i}{\mathrm{d}t} + Ri + K_{vr}\omega_r
\end{equation}
$$

donde $L$ y $R$ son la inductancia y la resistencia del bobinado, respectivamente, y $K_{vr}$ es la constante de velocidad del motor (relación entre velocidad de giro y fuerza contraelectromotriz), que en un motor DC coincide con la constante de par $K_{tr}$. Si introducimos este último hecho en la ecuación, junto con la relación de velocidades:

$$
\begin{equation}
    V = L\frac{\mathrm{d}i}{\mathrm{d}t} + Ri + nK_{tr}\omega_g
\end{equation}
$$

podemos deducir que la constante de velocidad equivalente, vista desde la salida de la reductora, es:

$$
\begin{equation}
    K_{vg} = nK_{tr} = \frac{K_{tg}}{\eta}
\end{equation}
$$

## Identificación de parámetros del motor [Pololu 2215](https://www.pololu.com/product/2215)

Con estas ecuaciones, podemos proceder a identificar los parámetros del motor. El fabricante proporciona, en la hoja de características del motor, unas curvas par-velocidad y par-corriente en régimen permanente para una tensión nominal de 6V, medidas en el eje de salida de la reductora. A partir de estas curvas, podemos obtener la mayoría de los parámetros del motor. Como los ensayos se han realizado en régimen permanente, los efectos de la inductancia $L$ y el momento de inercia equivalente del rotor $I_g$ no aparecen en los resultados, y por lo tanto no se pueden identificar. Si se eliminan estas dos constantes, las ecuaciones que sirven para extraer los parámetros de las curvas del fabricante serán:

$$
\begin{align}
    \tau_g &= K_{tg}i - b_g\omega_g \\
    6 &= Ri + K_{vg}\omega_g
\end{align}
$$

En estas ecuaciones aparecen las cuatro constantes que sí podremos identificar a partir de las curvas: $K_{tg}$, $K_{vg}$, $b_g$ y $R$.

Las curvas par-velocidad y par-corriente correspondientes a los ensayos del fabricante para el modelo 2215 son las siguientes:

$$
\begin{align}
    \omega_g &= 410 - 32\tau_g \\
    i &= 0.073 + 0.11\tau_g
\end{align}
$$

donde el par está en kg·mm, la velocidad en rpm, y la corriente en A. En estas dos ecuaciones hay cuatro constantes, así que si se cogen las dos ecuaciones del motor, y se expresan también en forma par-velocidad y par-corriente, se pueden identificar los cuatro parámetros, que deberán ser convertidos posteriormente a unidades del SI.

Las dos constantes que faltan se pueden obtener a partir de ensayos dinámicos, midiendo la respuesta a una entrada de voltaje determinada con una carga de inercia conocida. Respecto a la inductancia $L$, a partir de aquí ignoraremos su efecto, dado que la dinámica de la parte eléctrica es mucho más rápida que la de la parte mecánica, y no tiene un efecto relevante en el control. Para la inercia equivalente del rotor $I_g$ se usará el siguiente valor, estimado a partir de un ensayo básico:

$$
    I_g = 5·10^{-5} \mathrm{kg·m}^2
$$

## Efecto de la reductora adicional del robot

El robot que vamos a controlar tiene una reductora adicional de relación $n_r$ 41:25, así que si queremos obtener un modelo del sistema motor-reductora completo, deberemos aplicar una segunda conversión a los parámetros obtenidos para la primera reductora. En este caso, consideraremos que las pérdidas mecánicas de la segunda reductora son despreciables frente a las de la primera, que tiene una relación de velocidades mucho más elevada (38437:507):

$$
\begin{equation}
    K_{tw} = n_rK_{tg} \qquad
    K_{vw} = n_rK_{vg} \qquad
    b_w = n_r^2b_g     \qquad
    I_w = n_r^2I_g
\end{equation}
$$

De este modo, las ecuaciones del motor, considerando el par $\tau_w$ y la velocidad $\omega_w$ ya en la rueda, serán:

$$
\begin{align}
    \tau_w &= K_{tw}i - I_w\dot\omega_w - b_w\omega_w \\
    V &= Ri + K_{vw}\omega_w
\end{align}
$$

## Introducción del segundo motor

Hasta ahora hemos considerado un sólo motor, pero nuestro robot tiene dos ruedas, con un motor independiente en cada una. Para evitar tener que desarrollar un modelo completo en tres dimensiones, en la primera fase de diseño del controlador se simplificará el sistema, tratándolo como un modelo plano. Eso implica que el robot se moverá en línea recta, con las dos ruedas girando siempre a la misma velocidad $\omega_m$, mientras ambos motores reciben el mismo voltaje de entrada. Bajo estos supuestos, se puede considerar que el par total $\tau_m$ será dos veces el par de una sola rueda $\tau_w$. Como los motores van conectados en paralelo a la tensión de entrada $V$, la segunda ecuación no cambia:

$$
\begin{align}
    \tau_m &= 2K_{tw}i - 2I_w\dot\omega_m - 2b_w\omega_m \\
    V &= Ri + K_{vw}\omega_m
\end{align}
$$

En estas ecuaciones se pueden identificar los parámetros correspondientes al modelo completo, que será el utilizado para el diseño del controlador. Para mayor comodidad, se muestran ya los valores en función de los parámetros identificados previamente a la salida de la primera reductora:

$$
\begin{equation}
    K_t = 2n_rK_{tg} \qquad
    K_v = n_rK_{vg}  \qquad
    b = 2n_r^2b_g    \qquad
    I_m = 2n_r^2I_g
\end{equation}
$$

Así, las ecuaciones definitivas que usaremos para diseñar el controlador resultan:

$$
\begin{align}
    \tau_m &= K_ti - I_m\dot\omega_m - b\omega_m \\
    V &= Ri + K_v\omega_m
\end{align}
$$

## Funcionamiento como generador

Todo lo desarollado hasta ahora se basa en la suposición de que el motor produce una potencia positiva, que es transmitida a la rueda a través de la reductora. Pero cuando el motor está frenando, la potencia fluye en sentido contrario, de la rueda al motor. En esta situación, lo que antes era la salida de la reductora pasa a ser la entrada, y esto implica que las pérdidas de par se producirán en el lado opuesto. En general, en función del signo de la potencia mecánica, se cumplirá que:

$$
\begin{equation}
    \left\{
    \begin{matrix}
        \tau_g = \eta n\tau_r && \dot W > 0 \\
        \eta\tau_g = n\tau_r && \dot W < 0
    \end{matrix}
    \right.
\end{equation}
$$

Esto quiere decir que los parámetros del sistema serán diferentes dependiendo de si el motor está trabajando como generador o como motor. Es fácil demostrar que los parámetros del conjunto motor-reductora, cuando trabaja en modo generador, pasan a ser:

$$
\begin{equation}
    K_t^G = \frac{K_t}{\eta^2} \qquad
    K_v^G = K_v                \qquad
    b^G = \frac{b}{\eta^2}     \qquad
    I_m^G = \frac{I_m}{\eta^2}
\end{equation}
$$

donde el rendimiento de la primera reductora $\eta$ se puede obtener fácilmente dividiendo $K_{tg}$ entre $K_{vg}$.

Si los parámetros del modelo son diferentes en cada caso, lo correcto sería diseñar dos controladores, uno para modo motor y otro para modo generador. Durante el funcionamiento, habría que detectar qué está haciendo el motor en cada momento, y aplicar las ganancias del controlador correspondiente. Para conocer el signo de la potencia mecánica, basta con comparar el signo del par $\tau_m$ con el de la velocidad de giro $\omega_m$. Para poder estimar el par sin tener que medir la corriente, se puede despejar ésta en una ecuación y sustituir el resultado en la otra:

$$
\begin{equation}
    \tau_m = \frac{K_t}{R}V - \left(\frac{K_tK_v}{R} + b\right)\omega_m - I_m\dot\omega_m
\end{equation}
$$

Si hacemos lo mismo en modo generador, sustituyendo los valores correspondientes de las constantes, se demuestra que:

$$
\begin{equation}
    \tau_m^G = \frac{\tau_m}{\eta^2}
\end{equation}
$$

En nuestra aplicación, el efecto del rendimiento de la reductora no es muy relevante, así que no merece la pena todo el trabajo de diseñar dos controladores e implementar en el microcontrolador la detección del modo de funcionamiento. El controlador LQR es bastante robusto frente a errores en los parámetros, y se puede comprobar que las ganancias en modo motor y en modo generador son muy parecidas, así que se podrán utilizar los parámetros originales sin problema o, si se quiere afinar algo más, un valor promedio entre modo motor y modo generador.